In [1]:
import pandas as pd
import nltk
import numpy as np
import re 
import string
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier 
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import KFold, cross_val_score



In [ ]:
9

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\EL
[nltk_data]     SONNI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
 nltk.download('wordnet')


[nltk_data] Downloading package wordnet to C:\Users\EL
[nltk_data]     SONNI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
wn = nltk.WordNetLemmatizer()
stopwords=nltk.corpus.stopwords.words("english")

In [5]:
TRData=pd.read_csv("train_2kmZucJ.csv")
TSData=pd.read_csv("test_oJQbWVk.csv")
label=pd.read_csv("train_2kmZucJ.csv")
# fill missing values with the previous value
TRData.fillna(method='ffill', inplace=True)
TSData.fillna(method='ffill', inplace=True)
label.fillna(method='ffill', inplace=True)
# TRData["tweet"]=TRData["tweet;;;;;;;"]
# TRData.drop("tweet;;;;;;;",axis=1 ,inplace=True)


In [6]:

print("train_dataset--->",TRData.shape)
print("test_dataset--->",TSData.shape)
print("train_dataset--->",label.shape)


train_dataset---> (7920, 3)
test_dataset---> (1953, 2)
train_dataset---> (7920, 3)


In [7]:
TRData.columns


Index(['id', 'label', 'tweet'], dtype='object')

In [8]:
TRData=TRData.drop("label",axis=1)
## convertinig each text into lower case
TRData["tweet"]=TRData["tweet"].str.lower()

In [9]:
train=TRData
train.head()

,id,tweet
0,1,#fingerprint #pregnancy test https://goo.gl/h1...
1,2,finally a transparant silicon case ^^ thanks t...
2,3,we love this! would you go? #talk #makememorie...
3,4,i'm wired i know i'm george i was made that wa...
4,5,what amazing service! apple won't even talk to...


In [10]:
# TSData["tweet"]=TSData["tweet;;;;;"]
# TSData.drop("tweet;;;;;",axis=1 ,inplace=True)
test=TSData
test.head()


,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [11]:
final_dataset=pd.concat([train,test],axis=0)

final_dataset.head()


,id,tweet
0,1,#fingerprint #pregnancy test https://goo.gl/h1...
1,2,finally a transparant silicon case ^^ thanks t...
2,3,we love this! would you go? #talk #makememorie...
3,4,i'm wired i know i'm george i was made that wa...
4,5,what amazing service! apple won't even talk to...


In [12]:
final_dataset["tweet_length"]=final_dataset["tweet"].apply(lambda x:len(x)-x.count(" "))
train["tweet_length"]=train["tweet"].apply(lambda x:len(x)-x.count(" "))
test["tweet_length"]=test["tweet"].apply(lambda x:len(x)-x.count(" "))


In [13]:
def punct(text):
    count=sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text)-text.count(" ")),3)*100
final_dataset['punct%']=final_dataset["tweet"].apply(lambda x: punct(x))
train['punct%']=train["tweet"].apply(lambda x: punct(x))
test['punct%']=test["tweet"].apply(lambda x: punct(x))
final_dataset.head()

,id,tweet,tweet_length,punct%
0,1,#fingerprint #pregnancy test https://goo.gl/h1...,116,13.8
1,2,finally a transparant silicon case ^^ thanks t...,115,13.9
2,3,we love this! would you go? #talk #makememorie...,109,16.5
3,4,i'm wired i know i'm george i was made that wa...,96,17.7
4,5,what amazing service! apple won't even talk to...,102,4.9


In [14]:
final_dataset["tweet"] = final_dataset["tweet"].str.replace("[^a-z ]", "", regex=True)
train["tweet"] = train["tweet"].str.replace("[^a-z ]", "", regex=True)
test["tweet"] = test["tweet"].str.replace("[^a-z ]", "", regex=True)


In [15]:
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopwords]
    return text
# adding new column tweet_nostopwords which consists of tokenized tweets with no stopwords.
final_dataset['tweet_nostopwords'] = final_dataset['tweet'].apply(lambda x: clean_text(x.lower()))
train['tweet_nostopwords'] = train['tweet'].apply(lambda x: clean_text(x.lower()))
test['tweet_nostopwords'] = test['tweet'].apply(lambda x: clean_text(x.lower()))

final_dataset.head()

,id,tweet,tweet_length,punct%,tweet_nostopwords
0,1,fingerprint pregnancy test httpsgooglhmfqv and...,116,13.8,"[fingerprint, pregnancy, test, httpsgooglhmfqv..."
1,2,finally a transparant silicon case thanks to ...,115,13.9,"[finally, transparant, silicon, case, thanks, ..."
2,3,we love this would you go talk makememories un...,109,16.5,"[love, would, go, talk, makememories, unplug, ..."
3,4,im wired i know im george i was made that way ...,96,17.7,"[im, wired, know, im, george, made, way, iphon..."
4,5,what amazing service apple wont even talk to m...,102,4.9,"[amazing, service, apple, wont, even, talk, qu..."


In [16]:
# function for converting tokenized tweets into lemmatized tweets.
def lemmatizing(tokenized_text):
    text =" ".join (  [wn.lemmatize(word) for word in tokenized_text]  )
    return text

# adding new column tweet_lemmatized which consists of lemmatized tweets.
final_dataset['tweet_lemmatized'] = final_dataset['tweet_nostopwords'].apply(lambda x: lemmatizing(x))
train['tweet_lemmatized'] = train['tweet_nostopwords'].apply(lambda x: lemmatizing(x))
test['tweet_lemmatized'] = test['tweet_nostopwords'].apply(lambda x: lemmatizing(x))

final_dataset.head()


,id,tweet,tweet_length,punct%,tweet_nostopwords,tweet_lemmatized
0,1,fingerprint pregnancy test httpsgooglhmfqv and...,116,13.8,"[fingerprint, pregnancy, test, httpsgooglhmfqv...",fingerprint pregnancy test httpsgooglhmfqv and...
1,2,finally a transparant silicon case thanks to ...,115,13.9,"[finally, transparant, silicon, case, thanks, ...",finally transparant silicon case thanks uncle ...
2,3,we love this would you go talk makememories un...,109,16.5,"[love, would, go, talk, makememories, unplug, ...",love would go talk makememories unplug relax i...
3,4,im wired i know im george i was made that way ...,96,17.7,"[im, wired, know, im, george, made, way, iphon...",im wired know im george made way iphone cute d...
4,5,what amazing service apple wont even talk to m...,102,4.9,"[amazing, service, apple, wont, even, talk, qu...",amazing service apple wont even talk question ...


In [17]:
from sklearn.feature_extraction.text import CountVectorizer


In [18]:
count_vect=CountVectorizer(analyzer=clean_text)
X_counts=count_vect.fit_transform(final_dataset["tweet_lemmatized"])
print(X_counts.shape)

(9873, 25093)


In [19]:
X_counts=count_vect.fit_transform(train["tweet_lemmatized"])
ts_counts=count_vect.fit_transform(test["tweet_lemmatized"])


In [20]:
X_counts

<7920x20535 sparse matrix of type '<class 'numpy.int64'>'
	with 91176 stored elements in Compressed Sparse Row format>

In [21]:
X_counts_df=pd.DataFrame(X_counts.toarray())
ts_counts_df=pd.DataFrame(X_counts.toarray())

#X_counts_df.columns=count_vect.get_feature_names()
X_counts_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,20525,20526,20527,20528,20529,20530,20531,20532,20533,20534
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# adding the tweet length and punct% column to the  vectorized dataset
a=pd.DataFrame(np.array(final_dataset['tweet_length']).reshape(-1,1),columns=["tweet_length"])
b=pd.DataFrame(np.array(final_dataset['punct%']).reshape(-1,1),columns=["punct%"])
final_dataset1= pd.concat([a,b,X_counts_df],axis=1)

In [23]:
# adding the tweet length and punct% column to the  vectorized dataset
a=pd.DataFrame(np.array(train['tweet_length']).reshape(-1,1),columns=["tweet_length"])
b=pd.DataFrame(np.array(train['punct%']).reshape(-1,1),columns=["punct%"])
final_dataset1= pd.concat([a,b,X_counts_df],axis=1)

In [24]:
# adding the tweet length and punct% column to the  vectorized dataset
a=pd.DataFrame(np.array(test['tweet_length']).reshape(-1,1),columns=["tweet_length"])
b=pd.DataFrame(np.array(test['punct%']).reshape(-1,1),columns=["punct%"])
final_ts= pd.concat([a,b,X_counts_df],axis=1)

In [25]:
# final dataset for fitting ML model
final_dataset1.head()

,tweet_length,punct%,0,1,2,3,4,5,6,7,...,20525,20526,20527,20528,20529,20530,20531,20532,20533,20534
0,116,13.8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,115,13.9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,109,16.5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,96,17.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,102,4.9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# splitting the dataset into train and test as we specified above.
final_train=final_dataset1.iloc[0:7920,:]
final_test=final_dataset1.iloc[7920:,:]

In [27]:
# creating object of the model
rf = RandomForestClassifier(n_jobs=-1,n_estimators=300)

In [28]:
final_train.dropna(inplace=True)

In [29]:
# fitting the model
rf.fit(final_train,label['label'])

RandomForestClassifier(n_estimators=300, n_jobs=-1)

In [30]:
final_ts.isnull().sum()
final_ts.dropna(inplace=True)

In [31]:
# predicting on test dataset
y_pred=rf.predict(final_ts )
y_pred=pd.DataFrame(y_pred)
y_pred.head()


,0
0,0
1,0
2,0
3,0
4,1


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(analyzer=clean_text)
x_tfidf=tfidf.fit_transform(final_dataset["tweet_lemmatized"])
print(x_tfidf.shape)

(9873, 25093)


In [33]:
# in order to print the matrix we have to convert it to array
x_tfidf_df=pd.DataFrame(x_tfidf.toarray())
#x_tfidf_df.columns=tfidf.get_feature_names()
x_tfidf_df.head()

,0,1,2,3,4,5,6,7,8,9,...,25083,25084,25085,25086,25087,25088,25089,25090,25091,25092
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
# adding the tweet length and punct% column to the  vectorized dataset
a=pd.DataFrame(np.array(final_dataset['tweet_length']).reshape(-1,1))
b=pd.DataFrame(np.array(final_dataset['punct%']).reshape(-1,1))

In [35]:
final_dataset1= pd.concat([a,b,x_tfidf_df],axis=1)

In [36]:
# final dataset for fitting ML model
final_dataset1.head()

,0,0,0,1,2,3,4,5,6,7,...,25083,25084,25085,25086,25087,25088,25089,25090,25091,25092
0,116,13.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,115,13.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,109,16.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,96,17.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,102,4.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# splitting the dataset into train and test as we specified above.
final_train=final_dataset1.iloc[0:7920,:]
final_test=final_dataset1.iloc[7920:,:]

In [38]:
# creating object of the model
rf = RandomForestClassifier(n_jobs=-1,n_estimators=300)

In [39]:
# fitting the model
rf.fit(final_train,label['label'])

RandomForestClassifier(n_estimators=300, n_jobs=-1)

In [48]:
# predicting on test dataset
y_pred=rf.predict(final_test)
y_pred=pd.DataFrame(y_pred)
y_pred.head()

,0
0,1
1,0
2,1
3,1
4,1


In [49]:
print(y_pred.to_string())


      0
0     1
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    1
18    1
19    0
20    0
21    0
22    0
23    0
24    1
25    0
26    0
27    0
28    1
29    0
30    1
31    0
32    0
33    0
34    0
35    1
36    1
37    0
38    0
39    0
40    0
41    0
42    0
43    1
44    0
45    0
46    0
47    0
48    0
49    1
50    1
51    0
52    0
53    0
54    1
55    0
56    1
57    0
58    0
59    1
60    0
61    0
62    0
63    0
64    0
65    0
66    0
67    1
68    0
69    1
70    0
71    0
72    0
73    0
74    1
75    0
76    0
77    0
78    0
79    1
80    0
81    0
82    0
83    1
84    0
85    1
86    0
87    0
88    0
89    0
90    0
91    0
92    0
93    0
94    1
95    1
96    1
97    0
98    1
99    1
100   1
101   0
102   0
103   1
104   0
105   1
106   1
107   0
108   0
109   0
110   0
111   0
112   0
113   0
114   1
115   0
116   0
117   0
118   0
119   1
120   0
121   1
122   0
123   0


In [50]:
y_pred.index=range(7921,9874)
y_pred.columns=["label"]
# set the name of the index
y_pred.index.name = 'id'

In [51]:
print(y_pred.head())



      label
id         
7921      1
7922      0
7923      1
7924      1
7925      1


In [52]:
y_pred.to_csv('Sample Submession.csv')